7.Half-Close based on TCP
=======
---
### 1. Problems with one-way termination
### 2. Half-close of Socket
### 3. Shut-down for better termination
### 4. Termination of the output stream
### 5. Half-close based File Transfer Program

---
#### 1. Problems with one-way termination
7.1.1. Features of the funtions close and closesocket
- This means the complete destruction of the socket.
- Since the socket is completely destructed, there are no more input and output.
- No matter the state of relative socket, this is formed general termination.
- If the data transferring of relative host isn't complete, there can be problem.
- The half-close method is a solution of that problem.

#### 2. Half-close of Socket
7.2.1. Half-close
	- Half-close method is also called **beautiful termination**.
	- This is a way to terminate only one of the input or output streams.
	- The meaning of wanting termination is the situation that there are no more data that have to transfer. Therefore, the output stream can be teminated.
	- However, this situation is unchecked situation about what a relative socket want to transfer or not. So the output stream need not be turned on at the end.
	- **half-close means the turning off the input stream.**
	
#### 3. Shut-down for better termination
7.3.1. The funtion shutdown()
```c
#include <sys/socket.h>
int shutdown(int sock, int howto);
```
- Success : 0
- Failure : -1
- sock : socket ready to terminate
- howto : how to terminate
	1. SHUT_RD : terminating input stream
	2. SHUT_WR : terminating output stream
	3. SHUT_RDWR : terminating RDWR

If the function close() is called, EOF transfers to the relative socket. And EOF means termination of all of data transferring. This is one of purpose of calling the function close(). However, **when the output stream is terminated, EOF is sent.** Thus, it can replace the function close(), and wait the termination of the relative host.

#### 4. Termination of the output stream
1. If the output stream is terminated, EOF message is sent to connected host before.
2. EOF transferring can announce the end of data transferring.
3. When EOF is sent, the functions read(), and recv() of the relative host return 0.
 
### 5. Half-close based File Transfer Program
7.5.1. A part of file_server.c
```c
while (1) {
	read_cnt = fread((void*)buf, 1, BUF_SIZE, fp); // fp is FILE *.
	if (read_cnt < BUF_SIZE) {
		write(clnt_sd, buf, read_cnt);
		break;	
	}
	write(clnt_sd, buf, BUF_SIZE);
}
shutdown(clnt_sd, SHUT_WR);
read(clnt_sd, buf, BUF_SIZE); // Maybe it is nice that the hash value of file is sent.
printf("Message from client: %s \n", buf);
fclose(fp);
close(clnt_sd);
close(serv_sd);
```
7.5.1. A part of file_client.c
```c
while ((read_cnt = read(sd, buf, BUF_SIZE))!= 0)
	fwrite((void*)buf, 1, read_cnt, fp); // fp에 파일쓰기중
puts("Received file data");
write(sd, "Thank you", 10);
fclose(fp);
close(sd);
```
7.5.2. Full code
[file_server.c][https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch7/file_server.c](https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch7/file_server.c)
[file_client.c][https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch7/file_client.c](https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch7/file_client.c)